# import

In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from keras.models import Model
from keras.utils import np_utils
import keras 
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


## '''read data and preprocess'''

In [13]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 2000
nb_feature = 0
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

#flist  = ['MiddlePhalanxTW']
#for each in flist:
each = 'Adiac'

fname = each
x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')
nb_classes = len(np.unique(y_test))
nb_feature = x_train.shape[1]
batch_size = min(x_train.shape[0]/10, 16)

y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std) 
x_test = (x_test - x_train_mean)/(x_train_std)

x_train = x_train.reshape(x_train.shape + (1,1,))
x_test = x_test.reshape(x_test.shape + (1,1,))

In [14]:
print("# x_train.shape: ",x_train.shape)
print("# x_test.shape: ",x_test.shape)

print("# Y_train.shape: ",Y_train.shape)
print("# Y_test.shape: ",Y_test.shape)

# x_train.shape:  (390, 176, 1, 1)
# x_test.shape:  (391, 176, 1, 1)
# Y_train.shape:  (390, 37)
# Y_test.shape:  (391, 37)


# 类似VGG的卷积神经网络：

In [18]:
'''对比一下示例数据与所用数据的不同'''
x_ = np.random.random((10, 10, 10, 3))
y_ = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
print("# x_.shape: ",x_.shape)
print("# y_.shape: ",y_.shape)

# x_.shape:  (10, 10, 10, 3)
# y_.shape:  (100, 10)


array([[[[  4.89033496e-01,   8.87785390e-01,   8.01902942e-03],
         [  2.43661146e-01,   9.24611982e-02,   6.14745651e-01],
         [  8.56031750e-01,   9.38926254e-01,   5.69361455e-01],
         ..., 
         [  4.73535731e-01,   7.22420613e-01,   8.67720174e-01],
         [  1.08069015e-01,   5.33860024e-01,   2.15286169e-01],
         [  4.22824508e-01,   3.92013645e-01,   2.91933812e-01]],

        [[  6.81409662e-01,   8.72292199e-01,   2.05931954e-01],
         [  2.46154948e-01,   1.40822385e-01,   8.78539714e-01],
         [  3.19489163e-01,   6.42116469e-01,   3.30648718e-01],
         ..., 
         [  2.08748879e-01,   7.05215766e-01,   8.22010228e-01],
         [  5.25973012e-01,   1.32789936e-01,   4.92678853e-01],
         [  4.98027349e-01,   3.18630919e-01,   9.72358053e-02]],

        [[  6.52165385e-01,   6.50345148e-01,   5.50664296e-01],
         [  8.08081715e-01,   9.65816049e-01,   6.77025835e-01],
         [  8.33104566e-01,   4.65478167e-01,   4.750102

In [23]:
x_train.shape[1:]

(176, 1)

In [45]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

x = keras.layers.Input(x_train.shape[1:])
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.

# input: 391x176 images with 1 channels -> (176, 1, 1) tensors.
# this applies 32 convolution filters of size 3x3 each.


conv2 = keras.layers.Conv2D(128, 8, 1, border_mode='same')(x)
conv2 = keras.layers.normalization.BatchNormalization()(conv2)
conv2 = keras.layers.Activation('relu')(conv2)
#conv2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
drop_out = Dropout(0.25)(conv2)

conv3 = keras.layers.Conv2D(256, 5, 1, border_mode='same')(conv2)
conv4 = keras.layers.Conv2D(128, 3, 1, border_mode='same')(conv3)
conv4 = keras.layers.normalization.BatchNormalization()(conv4)
conv4 = keras.layers.Activation('relu')(conv4)
#conv4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
drop_out = Dropout(0.25)(conv4)

conv4 = keras.layers.Flatten()(conv4)
conv4 = keras.layers.Dense(128, activation='softmax')(conv4)
drop_out = Dropout(0.25)(conv4)
out = keras.layers.Dense(nb_classes, activation='softmax')(conv4)

model = Model(input=x, output=out)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])


hist = model.fit(x_train, Y_train, batch_size=16, epochs=10)
score = model.evaluate(x_test, Y_test, batch_size=16)

# Print the testing results which has the lowest training loss.
#log = pd.DataFrame(hist.history)
#print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (8, 1), padding="same")`
  app.launch_new_instance()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 1), padding="same")`
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 1), padding="same")`
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/10
390/390 [==============================] - 8s 19ms/step - loss: 3.6126 - acc: 0.0282
Epoch 2/10
390/390 [==============================] - 7s 17ms/step - loss: 3.6032 - acc: 0.0462
Epoch 3/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5948 - acc: 0.0410
Epoch 4/10
390/390 [==============================] - 6s 16ms/step - loss: 3.6037 - acc: 0.0308
Epoch 5/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5996 - acc: 0.0282
Epoch 6/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5966 - acc: 0.0308
Epoch 7/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5938 - acc: 0.0231
Epoch 8/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5912 - acc: 0.0333
Epoch 9/10
390/390 [==============================] - 6s 16ms/step - loss: 3.5897 - acc: 0.0385
Epoch 10/10
391/391 [==============================] - 2s 6ms/step


# CNN

In [15]:
import numpy as np
np.random.seed(1337)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

# 全局变量
batch_size = batch_size
nb_classses = nb_classes
epochs = nb_epochs
# input image dimensions
img_rows, img_cols = x_train.shape[1], x_train.shape[2]
# number of convolutional filters to use
nb_filters_1 = 128
nb_filters_2 = 256
nb_filters_3 = 128
# size of pooling area for max pooling
pool_size = (3,1)
# convolution kernel size
kernel_size_1 = (8,1)
kernel_size_2 = (5,1)
kernel_size_3 = (3,1)
input_shape = (x_train.shape[1:])
print(input_shape)

(176, 1, 1)


In [6]:
#构建模型
model = Sequential()  
""" 
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], 
                        border_mode='same', 
                        input_shape=input_shape)) 
"""  
model.add(Convolution2D(nb_filters_1, (kernel_size_1[0], kernel_size_1[1]),  
                        padding='same',  
                        input_shape=input_shape)) # 卷积层1  
model.add(Activation('relu')) #激活层  

model.add(Convolution2D(nb_filters_2, (kernel_size_2[0], kernel_size_2[1]))) #卷积层2  
model.add(Activation('relu')) #激活层

model.add(Convolution2D(nb_filters_3, (kernel_size_3[0], kernel_size_3[1]))) #卷积层3  
model.add(Activation('relu')) #激活层  


model.add(MaxPooling2D(pool_size=pool_size)) #池化层  
model.add(Dropout(0.25)) #神经元随机失活  
model.add(Flatten()) #拉成一维数据  
model.add(Dense(128)) #全连接层1  
model.add(Activation('relu')) #激活层  
model.add(Dropout(0.5)) #随机失活  
model.add(Dense(nb_classes)) #全连接层2  
model.add(Activation('softmax')) #Softmax评分  
  
#编译模型  
model.compile(loss='categorical_crossentropy',  
              optimizer='adadelta',  
              metrics=['accuracy'])  

In [16]:
#训练模型  
hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=epochs,  
          verbose=1, validation_data=(x_test, Y_test))  
#评估模型  
score = model.evaluate(x_test, Y_test, verbose=0)  
print('Test score:', score[0])  
print('Test accuracy:', score[1])  
#Print the testing results which has the lowest training loss.
log = pd.DataFrame(hist.history)
print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

Train on 390 samples, validate on 391 samples
Epoch 1/2000
390/390 [==============================] - 6s - loss: 3.6026 - acc: 0.0256 - val_loss: 3.6169 - val_acc: 0.0435
Epoch 2/2000
390/390 [==============================] - 6s - loss: 3.6002 - acc: 0.0333 - val_loss: 3.6211 - val_acc: 0.0409
Epoch 3/2000
390/390 [==============================] - 6s - loss: 3.6018 - acc: 0.0385 - val_loss: 3.6148 - val_acc: 0.0588
Epoch 4/2000
390/390 [==============================] - 6s - loss: 3.5900 - acc: 0.0564 - val_loss: 3.6119 - val_acc: 0.0665
Epoch 5/2000
390/390 [==============================] - 6s - loss: 3.5783 - acc: 0.0590 - val_loss: 3.5968 - val_acc: 0.0665
Epoch 6/2000
390/390 [==============================] - 6s - loss: 3.5705 - acc: 0.0590 - val_loss: 3.5805 - val_acc: 0.0537
Epoch 7/2000
390/390 [==============================] - 6s - loss: 3.5441 - acc: 0.0821 - val_loss: 3.5472 - val_acc: 0.0665
Epoch 8/2000
390/390 [==============================] - 5s - loss: 3.5418 - acc

390/390 [==============================] - 5s - loss: 2.1770 - acc: 0.3513 - val_loss: 2.3083 - val_acc: 0.3325
Epoch 66/2000
390/390 [==============================] - 5s - loss: 2.1541 - acc: 0.3410 - val_loss: 2.2918 - val_acc: 0.3095
Epoch 67/2000
390/390 [==============================] - 6s - loss: 2.1797 - acc: 0.3564 - val_loss: 2.3240 - val_acc: 0.3223
Epoch 68/2000
390/390 [==============================] - 6s - loss: 2.1380 - acc: 0.3590 - val_loss: 2.2230 - val_acc: 0.3504
Epoch 69/2000
390/390 [==============================] - 6s - loss: 2.0861 - acc: 0.3718 - val_loss: 2.2835 - val_acc: 0.3504
Epoch 70/2000
390/390 [==============================] - 5s - loss: 2.1532 - acc: 0.3436 - val_loss: 2.2413 - val_acc: 0.3274
Epoch 71/2000
390/390 [==============================] - 5s - loss: 2.1434 - acc: 0.3487 - val_loss: 2.2161 - val_acc: 0.3657
Epoch 72/2000
390/390 [==============================] - 6s - loss: 2.0876 - acc: 0.3538 - val_loss: 2.2487 - val_acc: 0.3376
Epoch 

390/390 [==============================] - 4s - loss: 1.4891 - acc: 0.4949 - val_loss: 1.8149 - val_acc: 0.4731
Epoch 130/2000
390/390 [==============================] - 4s - loss: 1.5819 - acc: 0.4769 - val_loss: 1.8140 - val_acc: 0.4629
Epoch 131/2000
390/390 [==============================] - 4s - loss: 1.5962 - acc: 0.4846 - val_loss: 1.8346 - val_acc: 0.4271
Epoch 132/2000
390/390 [==============================] - 4s - loss: 1.6060 - acc: 0.5000 - val_loss: 1.7032 - val_acc: 0.5371
Epoch 133/2000
390/390 [==============================] - 4s - loss: 1.5045 - acc: 0.5308 - val_loss: 1.7037 - val_acc: 0.5141
Epoch 134/2000
390/390 [==============================] - 4s - loss: 1.6022 - acc: 0.4795 - val_loss: 1.7147 - val_acc: 0.4808
Epoch 135/2000
390/390 [==============================] - 4s - loss: 1.4733 - acc: 0.5282 - val_loss: 1.6794 - val_acc: 0.5217
Epoch 136/2000
390/390 [==============================] - 4s - loss: 1.5139 - acc: 0.5385 - val_loss: 1.7402 - val_acc: 0.4757

390/390 [==============================] - 4s - loss: 1.1773 - acc: 0.6000 - val_loss: 1.5033 - val_acc: 0.5729
Epoch 194/2000
390/390 [==============================] - 4s - loss: 1.2229 - acc: 0.6026 - val_loss: 1.4267 - val_acc: 0.6061
Epoch 195/2000
390/390 [==============================] - 4s - loss: 1.2265 - acc: 0.6051 - val_loss: 1.4827 - val_acc: 0.6164
Epoch 196/2000
390/390 [==============================] - 4s - loss: 1.1066 - acc: 0.6462 - val_loss: 1.3783 - val_acc: 0.6343
Epoch 197/2000
390/390 [==============================] - 5s - loss: 1.1895 - acc: 0.6103 - val_loss: 1.4354 - val_acc: 0.6189
Epoch 198/2000
390/390 [==============================] - 4s - loss: 1.1856 - acc: 0.6179 - val_loss: 1.3903 - val_acc: 0.6394
Epoch 199/2000
390/390 [==============================] - 4s - loss: 1.1636 - acc: 0.6282 - val_loss: 1.4053 - val_acc: 0.6240
Epoch 200/2000
390/390 [==============================] - 4s - loss: 1.1367 - acc: 0.6179 - val_loss: 1.7912 - val_acc: 0.4859

390/390 [==============================] - 4s - loss: 0.9163 - acc: 0.6795 - val_loss: 1.4314 - val_acc: 0.6471
Epoch 258/2000
390/390 [==============================] - 4s - loss: 0.9053 - acc: 0.6872 - val_loss: 1.3439 - val_acc: 0.6240
Epoch 259/2000
390/390 [==============================] - 4s - loss: 1.0170 - acc: 0.6667 - val_loss: 1.3994 - val_acc: 0.6701
Epoch 260/2000
390/390 [==============================] - 4s - loss: 0.9390 - acc: 0.6744 - val_loss: 1.4855 - val_acc: 0.6164
Epoch 261/2000
390/390 [==============================] - 4s - loss: 0.9916 - acc: 0.6590 - val_loss: 1.3934 - val_acc: 0.6675
Epoch 262/2000
390/390 [==============================] - 4s - loss: 0.8987 - acc: 0.6769 - val_loss: 1.3366 - val_acc: 0.6675
Epoch 263/2000
390/390 [==============================] - 5s - loss: 0.9232 - acc: 0.6615 - val_loss: 1.3484 - val_acc: 0.6854
Epoch 264/2000
390/390 [==============================] - 4s - loss: 0.9133 - acc: 0.7026 - val_loss: 1.4279 - val_acc: 0.6471

...

390/390 [==============================] - 4s - loss: 0.1619 - acc: 0.9410 - val_loss: 1.6038 - val_acc: 0.7494
Epoch 1730/2000
390/390 [==============================] - 4s - loss: 0.1163 - acc: 0.9487 - val_loss: 1.5125 - val_acc: 0.7570
Epoch 1731/2000
390/390 [==============================] - 4s - loss: 0.1384 - acc: 0.9385 - val_loss: 1.5345 - val_acc: 0.7596
Epoch 1732/2000
390/390 [==============================] - 4s - loss: 0.1288 - acc: 0.9641 - val_loss: 1.6405 - val_acc: 0.7468
Epoch 1733/2000
390/390 [==============================] - 4s - loss: 0.1433 - acc: 0.9436 - val_loss: 1.5938 - val_acc: 0.7673
Epoch 1734/2000
390/390 [==============================] - 4s - loss: 0.1098 - acc: 0.9667 - val_loss: 1.9328 - val_acc: 0.7110
Epoch 1735/2000
390/390 [==============================] - 5s - loss: 0.1453 - acc: 0.9462 - val_loss: 1.6395 - val_acc: 0.7621
Epoch 1736/2000
390/390 [==============================] - 4s - loss: 0.1194 - acc: 0.9590 - val_loss: 1.6216 - val_acc:

390/390 [==============================] - 5s - loss: 0.1133 - acc: 0.9564 - val_loss: 1.5757 - val_acc: 0.7391
Epoch 1794/2000
390/390 [==============================] - 4s - loss: 0.1512 - acc: 0.9487 - val_loss: 1.6259 - val_acc: 0.7545
Epoch 1795/2000
390/390 [==============================] - 4s - loss: 0.1658 - acc: 0.9359 - val_loss: 1.5316 - val_acc: 0.7417
Epoch 1796/2000
390/390 [==============================] - 4s - loss: 0.1360 - acc: 0.9513 - val_loss: 1.6916 - val_acc: 0.7442
Epoch 1797/2000
390/390 [==============================] - 4s - loss: 0.1281 - acc: 0.9564 - val_loss: 1.6783 - val_acc: 0.7519
Epoch 1798/2000
390/390 [==============================] - 4s - loss: 0.1676 - acc: 0.9385 - val_loss: 1.7234 - val_acc: 0.7570
Epoch 1799/2000
390/390 [==============================] - 4s - loss: 0.1135 - acc: 0.9615 - val_loss: 1.6282 - val_acc: 0.7545
Epoch 1800/2000
390/390 [==============================] - 4s - loss: 0.1036 - acc: 0.9667 - val_loss: 1.7162 - val_acc:

390/390 [==============================] - 4s - loss: 0.1699 - acc: 0.9564 - val_loss: 1.6024 - val_acc: 0.7442
Epoch 1858/2000
390/390 [==============================] - 4s - loss: 0.1471 - acc: 0.9513 - val_loss: 1.6457 - val_acc: 0.7749
Epoch 1859/2000
390/390 [==============================] - 4s - loss: 0.1578 - acc: 0.9487 - val_loss: 1.6384 - val_acc: 0.7724
Epoch 1860/2000
390/390 [==============================] - 4s - loss: 0.0909 - acc: 0.9744 - val_loss: 1.5813 - val_acc: 0.7647
Epoch 1861/2000
390/390 [==============================] - 4s - loss: 0.1009 - acc: 0.9718 - val_loss: 1.7884 - val_acc: 0.7545
Epoch 1862/2000
390/390 [==============================] - 4s - loss: 0.1061 - acc: 0.9590 - val_loss: 1.6294 - val_acc: 0.7596
Epoch 1863/2000
390/390 [==============================] - 4s - loss: 0.1015 - acc: 0.9692 - val_loss: 1.8897 - val_acc: 0.7110
Epoch 1864/2000
390/390 [==============================] - 4s - loss: 0.1594 - acc: 0.9538 - val_loss: 1.5948 - val_acc:

390/390 [==============================] - 5s - loss: 0.1207 - acc: 0.9590 - val_loss: 1.7289 - val_acc: 0.7289
Epoch 1922/2000
390/390 [==============================] - 4s - loss: 0.1270 - acc: 0.9462 - val_loss: 1.5905 - val_acc: 0.7903
Epoch 1923/2000
390/390 [==============================] - 5s - loss: 0.1155 - acc: 0.9667 - val_loss: 1.5868 - val_acc: 0.7596
Epoch 1924/2000
390/390 [==============================] - 4s - loss: 0.1717 - acc: 0.9538 - val_loss: 1.5744 - val_acc: 0.7724
Epoch 1925/2000
390/390 [==============================] - 4s - loss: 0.1133 - acc: 0.9718 - val_loss: 1.6565 - val_acc: 0.7724
Epoch 1926/2000
390/390 [==============================] - 4s - loss: 0.1035 - acc: 0.9769 - val_loss: 1.5986 - val_acc: 0.7826
Epoch 1927/2000
390/390 [==============================] - 4s - loss: 0.0832 - acc: 0.9564 - val_loss: 1.5170 - val_acc: 0.7673
Epoch 1928/2000
390/390 [==============================] - 4s - loss: 0.1570 - acc: 0.9462 - val_loss: 1.5876 - val_acc:

390/390 [==============================] - 4s - loss: 0.0714 - acc: 0.9692 - val_loss: 1.6847 - val_acc: 0.7775
Epoch 1986/2000
390/390 [==============================] - 4s - loss: 0.1145 - acc: 0.9615 - val_loss: 1.6489 - val_acc: 0.7801
Epoch 1987/2000
390/390 [==============================] - 4s - loss: 0.1829 - acc: 0.9282 - val_loss: 1.6322 - val_acc: 0.7647
Epoch 1988/2000
390/390 [==============================] - 5s - loss: 0.1059 - acc: 0.9692 - val_loss: 1.5249 - val_acc: 0.7698
Epoch 1989/2000
390/390 [==============================] - 4s - loss: 0.1231 - acc: 0.9615 - val_loss: 1.6321 - val_acc: 0.7647
Epoch 1990/2000
390/390 [==============================] - 5s - loss: 0.1149 - acc: 0.9641 - val_loss: 1.5858 - val_acc: 0.7673
Epoch 1991/2000
390/390 [==============================] - 4s - loss: 0.1503 - acc: 0.9590 - val_loss: 1.5686 - val_acc: 0.7724
Epoch 1992/2000
390/390 [==============================] - 5s - loss: 0.1304 - acc: 0.9436 - val_loss: 1.7218 - val_acc:

In [19]:
#result = log.append(['min_loss',log.loc[log['loss'].idxmin]['loss'] ],['val_acc',log.loc[log['loss'].idxmin]['val_acc']])
s = pd.Series({'acc':"min_loss", 'loss':log.loc[log['loss'].idxmin]['loss'], 'lr':"val_acc",'val_acc':log.loc[log['loss'].idxmin]['val_acc']})
# 这里 Series 必须是 dict-like 类型
result = log.append(s, ignore_index=True)
# 这里必须选择ignore_index=True 或者给 Series 一个index值
result.to_csv(u'E:/Jupyter File/Jupyter notebook 2/UCR_Time_Series_Classification_Deep_Learning_Baseline-master/不同形式的网络的比较/Result_UCR_CNN.csv')
result

,acc,loss,val_acc,val_loss,lr
0,0.025641,3.602604,0.043478,3.616946,NaN
1,0.0333333,3.600230,0.040921,3.621115,NaN
2,0.0384615,3.601849,0.058824,3.614787,NaN
3,0.0564103,3.590040,0.066496,3.611929,NaN
4,0.0589744,3.578311,0.066496,3.596829,NaN
5,0.0589744,3.570452,0.053708,3.580519,NaN
6,0.0820513,3.544060,0.066496,3.547173,NaN
7,0.0615385,3.541821,0.063939,3.492329,NaN
8,0.0794872,3.497271,0.074169,3.463384,NaN
9,0.0974359,3.462606,0.081841,3.433763,NaN
